In [1]:
import torch
import torch.nn.functional as F
%pip install torch_geometric
from torch_geometric.nn import SAGEConv
import os
import json
import os
import pickle
import json
import random
import numpy as np
from torch_geometric.data import Data
import networkx as nx
import warnings
import math
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from torch.utils.data.dataloader import default_collate
from torch.utils.data import random_split

Note: you may need to restart the kernel to use updated packages.


In [2]:
def get_files_in_folder(input_folder):
    file_list = []
    for file_name in os.listdir(input_folder):
        file_path = os.path.join(input_folder, file_name)
        if os.path.isfile(file_path):
            file_list.append(file_path)
    return file_list

# Example usage:
folder_path = 'done'
verilog_files = get_files_in_folder(folder_path)
print(len(verilog_files))
print(verilog_files)

387
['done\\adder10_synth.txt', 'done\\adder11_synth.txt', 'done\\adder12_synth.txt', 'done\\adder13_synth.txt', 'done\\adder14_synth.txt', 'done\\adder15_synth.txt', 'done\\adder16_synth.txt', 'done\\adder17_synth.txt', 'done\\adder18_synth.txt', 'done\\adder19_synth.txt', 'done\\adder1_synth.txt', 'done\\adder20_synth.txt', 'done\\adder21_synth.txt', 'done\\adder22_synth.txt', 'done\\adder23_synth.txt', 'done\\adder24_synth.txt', 'done\\adder25_synth.txt', 'done\\adder26_synth.txt', 'done\\adder27_synth.txt', 'done\\adder28_synth.txt', 'done\\adder2_synth.txt', 'done\\adder3_synth.txt', 'done\\adder4_synth.txt', 'done\\adder5_synth.txt', 'done\\adder6_synth.txt', 'done\\adder7_synth.txt', 'done\\adder8_synth.txt', 'done\\adder9_synth.txt', 'done\\and10_gate_synth.txt', 'done\\and11_gate_synth.txt', 'done\\and12_gate_synth.txt', 'done\\and13_synth.txt', 'done\\and14_synth.txt', 'done\\and15_synth.txt', 'done\\and16_synth.txt', 'done\\and17_synth.txt', 'done\\and18_gate_synth.txt', 'do

In [3]:
def extracting_attributes(verilog_file):
    try:
        if os.path.isfile(verilog_file):
            with open(verilog_file, "r") as file:
                loaded_data = json.load(file)
                nodes = loaded_data[0]
                edges = loaded_data[1]
                label = loaded_data[2]
                
                x = torch.tensor(nodes, dtype=torch.float)
                edge_index = torch.tensor(edges, dtype=torch.long)
                y = torch.tensor(label, dtype=torch.float)
                num_nodes = x.size(0)
                
                # Create batch assignment vector (assuming one graph per file)
                batch = torch.zeros(num_nodes, dtype=torch.long)
                data = Data(x=x, edge_index=edge_index, y = y, batch = batch)
                return data
    
    except Exception as e:
        print(e)
        return e


In [4]:
class VerilogDataset(Dataset):  # Using Dataset from torch_geometric
    def __init__(self, verilog_files):
        print(f"Loaded {len(verilog_files)} Verilog files.")
        self.verilog_files = verilog_files

    def __len__(self):
        return len(self.verilog_files)

    def __getitem__(self, idx):
        verilog_file = self.verilog_files[idx]
        data = extracting_attributes(verilog_file)
        return data

dataset = VerilogDataset(verilog_files)
print(len(dataset))

Loaded 387 Verilog files.
387


In [5]:
print(dataset[0])
print(verilog_files[0])
print(dataset.verilog_files[0])

Data(x=[54, 7], edge_index=[2, 72], y=[1, 15], batch=[54])
done\adder10_synth.txt
done\adder10_synth.txt


In [6]:
def are_all_data_objects_unique(dataset):
    data_objects = []
    for data in dataset:
        if data in data_objects:
            return False
        data_objects.append(data)
    return True

# Example usage:
is_unique = are_all_data_objects_unique(dataset)
if is_unique:
    print("All data objects are unique.")
else:
    print("Duplicate data objects found.")


All data objects are unique.


In [7]:
y_labels = []
for data in dataset:
    # print(data)
    # print(data.y.tolist())
    y_labels.append(np.argmax(data.y.tolist()))

In [8]:
def custom_collate(batch):
    if isinstance(batch[0], Data):
        return batch
    else:
        return default_collate(batch)
    


In [9]:
X_train, X_test, y_train, y_test = train_test_split(dataset, y_labels, test_size=0.2, stratify = y_labels, random_state=41)
train_loader = DataLoader(X_train, batch_size=16, shuffle=True, collate_fn=custom_collate)
test_loader = DataLoader(X_test, batch_size=16, shuffle = False, collate_fn=custom_collate)

In [10]:
# len(train_loader.dataset)
print(train_loader.dataset[0])

Data(x=[57, 7], edge_index=[2, 69], y=[1, 15], batch=[57])


In [11]:
loader_iter = iter(train_loader)
batch = next(loader_iter)
# print(batch)
# print(batch.num_graphs)

In [17]:
import torch
import torch.nn as nn
from torch_geometric.nn import MessagePassing, global_mean_pool

class FunctionalGNN(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(FunctionalGNN, self).__init__(aggr='mean')  # or other aggregations like 'max' or 'sum'
        self.lin = nn.Linear(in_channels, out_channels)
    
    def forward(self, x, edge_index):
        return self.propagate(edge_index, x=x)
    
    def message(self, x_j):
        # x_j contains the node features of neighbors
        return x_j
    
    def update(self, aggr_out):
        # Apply a transformation to the aggregated messages
        return self.lin(aggr_out)

class NetlistModel(nn.Module):
    def __init__(self, num_node_features, num_classes):
        super(NetlistModel, self).__init__()
        self.conv1 = FunctionalGNN(num_node_features, 64)
        self.conv2 = FunctionalGNN(64, num_classes)
    
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = nn.ReLU()(x)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch = None)
        return nn.LogSoftmax(dim=1)(x)


In [19]:
from torch_geometric.data import DataLoader

num_node_features = 7
num_classes = 15
# Assuming you have a dataset of netlists
train_loader = DataLoader(X_train, batch_size=1, shuffle=True)
model = NetlistModel(num_node_features, num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(100):
    model.train()
    for data in train_loader:
        # print(data)
        optimizer.zero_grad()
        out = model(data)
        # print(out)
        # print(data.y)
        loss = loss_fn(out, data.y)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch}, Loss: {loss}")


Epoch 0, Loss: 2.6034250259399414
Epoch 1, Loss: 0.6033401489257812
Epoch 2, Loss: 5.123282432556152
Epoch 3, Loss: 1.8114774227142334
Epoch 4, Loss: 1.9421035051345825
Epoch 5, Loss: 1.8711310625076294
Epoch 6, Loss: 0.6114148497581482
Epoch 7, Loss: 1.3488481044769287
Epoch 8, Loss: 0.784159243106842
Epoch 9, Loss: 1.988486409187317
Epoch 10, Loss: 0.8302208185195923
Epoch 11, Loss: 2.229836940765381
Epoch 12, Loss: 0.4226386845111847
Epoch 13, Loss: 0.7152817249298096
Epoch 14, Loss: 0.4947541058063507
Epoch 15, Loss: 1.3369243144989014
Epoch 16, Loss: 0.6345152258872986
Epoch 17, Loss: 1.3860591650009155
Epoch 18, Loss: 0.03320732340216637
Epoch 19, Loss: 1.4694687128067017
Epoch 20, Loss: 2.4377429485321045
Epoch 21, Loss: 0.43617990612983704
Epoch 22, Loss: 1.7916841506958008
Epoch 23, Loss: 0.7586542963981628
Epoch 24, Loss: 0.3030214309692383
Epoch 25, Loss: 1.1925098896026611
Epoch 26, Loss: 1.203300952911377
Epoch 27, Loss: 3.6402111053466797
Epoch 28, Loss: 0.737667679786682

In [21]:
model.eval()
correct = 0
for data in X_train:
    out = model(data)  
    pred = out.argmax(dim=1)  # Use the class with highest probability.
    y_label = (data.y.tolist())
    y_label = y_label[0].index(1.0)
    pred_label = (pred.tolist())[0]
    # print(pred_label)
    # print(y_label)
    if y_label == pred_label:
        correct += 1            
    # correct += int((pred == data.y).sum())  # Check against ground-truth labels.
train_acc = correct / len(X_train)  # Derive ratio of correct predictions.

train_acc

0.6245954692556634

In [22]:
model.eval()
correct = 0
for data in X_test:
    out = model(data)  
    pred = out.argmax(dim=1)  # Use the class with highest probability.
    y_label = (data.y.tolist())
    y_label = y_label[0].index(1.0)
    pred_label = (pred.tolist())[0]
    # print(pred_label)
    # print(y_label)
    if y_label == pred_label:
        correct += 1            
    # correct += int((pred == data.y).sum())  # Check against ground-truth labels.
test_acc = correct / len(X_test)  # Derive ratio of correct predictions.

test_acc

0.5769230769230769

In [23]:
torch.save(model.state_dict(), 'GNN62_58_100.pth')

In [27]:
print(dataset[0].y)
print(dataset[10].y)
print(dataset[0].y.tolist() == dataset[10].y.tolist())

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
True


In [28]:
import random

def generate_positive_negative_pairs(dataset):
    positive_pairs = []
    negative_pairs = []
    for i, data_i in enumerate(dataset):
        for j, data_j in enumerate(dataset):
            if i != j:
                if data_i.y.tolist() == data_j.y.tolist():  # Define your own similarity function
                    positive_pairs.append((data_i, data_j))
                else:
                    negative_pairs.append((data_i, data_j))
    return positive_pairs, negative_pairs


In [29]:
class ContrastiveLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin
    
    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2)
        loss_contrastive = torch.mean((1 - label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))
        return loss_contrastive


In [50]:
positive_pairs, negative_pairs = generate_positive_negative_pairs(X_train)
len(positive_pairs)

6354

In [51]:
len(negative_pairs)

88818

In [52]:
total_pairs = positive_pairs + negative_pairs
print(len(total_pairs))

95172


In [53]:
abs(3-4)

1

In [55]:
import torch.nn.functional as F

# Assuming you have a dataset of netlists
# train_loader = DataLoader(X_train, batch_size=1, shuffle=True)
model2 = NetlistModel(num_node_features, num_classes)
optimizer = torch.optim.Adam(model2.parameters(), lr=0.01)
contrastive_loss_fn = ContrastiveLoss(margin=0.5)

# Generate positive and negative pairs

prev = 0
for epoch in range(100):
    model2.train()
    total_loss = 0
    for (data_i, data_j) in total_pairs:
        # print(c)
        # c+=1 
        optimizer.zero_grad()
        out_i = model2(data_i)
        out_j = model2(data_j)
        # Label is 1 for positive pairs and 0 for negative pairs
        label = 1 if (data_i, data_j) in positive_pairs else 0
        label = torch.tensor([label], dtype=torch.float32).to(out_i.device)
        loss = contrastive_loss_fn(out_i, out_j, label)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    if abs(prev - total_loss) < 0.0001:
        break
    prev = total_loss
    print(f'Epoch {epoch+1}, Loss: {total_loss / len(total_pairs)}')


Epoch 1, Loss: 0.022616562414411578
Epoch 2, Loss: 0.01669057390476685
Epoch 3, Loss: 0.016690571693539026
Epoch 4, Loss: 0.01669056635410213


KeyboardInterrupt: 

In [56]:
model2.eval()
correct = 0
for data in X_train:
    out = model2(data)  
    pred = out.argmax(dim=1)  # Use the class with highest probability.
    y_label = (data.y.tolist())
    y_label = y_label[0].index(1.0)
    pred_label = (pred.tolist())[0]
    # print(pred_label)
    # print(y_label)
    if y_label == pred_label:
        correct += 1            
    # correct += int((pred == data.y).sum())  # Check against ground-truth labels.
test_acc = correct / len(X_train)  # Derive ratio of correct predictions.

test_acc

0.07766990291262135